# Databricks SELECT Bug

This notebook showcases a databricks bug where selects do not work.

The first cell has some code taken from the the databricks SAT (Security Analysis Tool) where the issue was noticed.

The example has been shrunk down to the minimum code to get a failing result. We can see when creating the table, that the workspace_status column has a value of "RUNNING", however we are unable to search on "RUNNING".
It appears that there is a hidden character at the start of RUNNING, however when we obtain the cell, it's value is definitely RUNNING.

In [0]:
import json
import pandas as pd

from pyspark.sql.types import StructType
from pyspark.sql.functions import col, schema_of_json, from_json, concat_ws, collect_list, collect_set, explode
from collections import OrderedDict

def process_json_schema(df):
    df_with_schemas = df.select(schema_of_json(col("json_string")).alias("schema")).distinct()

    all_fields = OrderedDict()

    for row in df_with_schemas.select("schema").collect():
        schema_str = row.schema
        inner_schema = schema_str[7:-1] # Remove the outer 'STRUCT<' and '>' 
        schema = StructType.fromDDL(inner_schema)        
        for field in schema.fields:
            all_fields[field.name] = field
    final_struct = StructType(list(all_fields.values()))
    return final_struct
def bootstrap(viewname, func, **kwargs):
    apiDF = None
    try:
        lst = func(**kwargs)
        if lst:
            lstjson = [json.dumps(ifld) for ifld in lst]
            apiDF = spark.createDataFrame([(x,) for x in lstjson], ["json_string"])
            # display(apiDF)
            # Parse the JSON strings using the schema string
            apiDF = apiDF.select(from_json(col("json_string"), process_json_schema(apiDF)).alias("data")).select("data.*")
        else:
            # apiDF = spark.createDataFrame([], StructType([]))
            print("No Results!")
        if len(apiDF.take(1)) > 0:
            apiDF.write.option("delta.columnMapping.mode", "name").mode("overwrite").saveAsTable(viewname)
            print(f"Table created: `{viewname}`")
    except Exception:
        print("Exception encountered")
    return apiDF

In [0]:
%sql
use euw1_sandbox_playground.kuba_jasko -- Replace with any database name

In [0]:
def check_table(table):
    print(f"CHECKING: {table}")
    for where in ["", "= 'RUNNING'", "like 'RUNNING'", "like 'RUNNING%'", "like '%RUNNING'"]:
        sql = f"SELECT * FROM {table}"
        if where:
            sql += f" WHERE workspace_status {where}"
        df = spark.sql(sql)
        print(f" WHERE workspace_status {where}: {len(df.collect())}")

In [0]:
# Here we run the sample code with a minumum example input to reproduce the error
# We see that the field doesn't appear to be "RUNNING", but has some sort of prefix which is not visible
def get_content():
    return [
        {
            "workspace_info": {
                "compliance_security_profile": {
                    "is_enabled": False
                },
                "enhanced_security_monitoring": {
                    "is_enabled": False
                }
            },
            "workspace_status": "RUNNING",
            "workspace_status_message": "Workspace is running."
        },
    ]
table_name = "kuba_bootstrap"
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
df = bootstrap(table_name, get_content)
check_table(table_name)
# Just valiate with spark/python that the cell value is definitely "RUNNING"
running_cell = spark.table(table_name).collect()[0].workspace_status
assert running_cell == "RUNNING"
assert len(running_cell) == len("RUNNING")
spark.table(table_name).display()

Table created: `kuba_bootstrap`
CHECKING: kuba_bootstrap
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 0
 WHERE workspace_status like 'RUNNING': 0
 WHERE workspace_status like 'RUNNING%': 0
 WHERE workspace_status like '%RUNNING': 1


workspace_info,workspace_status,workspace_status_message
"List(List(false), List(false))",RUNNING,Workspace is running.


In [0]:
# If we check the original df, it works fine
df.filter("workspace_status = 'RUNNING'").display()
# If we save the df to a table, it breaks
spark.sql(f"DROP TABLE IF EXISTS {table_name_direct_write}")
table_name_direct_write = f"{table_name}_direct_write"
df.write.option("delta.columnMapping.mode", "name").mode("overwrite").saveAsTable(table_name_direct_write)
spark.table(table_name_direct_write).filter("workspace_status = 'RUNNING'").display()
check_table(table_name_direct_write)
# If we save again (overwrite) it works
df.write.option("delta.columnMapping.mode", "name").mode("overwrite").saveAsTable(table_name_direct_write)
spark.table(table_name_direct_write).filter("workspace_status = 'RUNNING'").display()
check_table(table_name_direct_write)

workspace_info,workspace_status,workspace_status_message
"List(List(false), List(false))",RUNNING,Workspace is running.


workspace_info,workspace_status,workspace_status_message


CHECKING: kuba_bootstrap_direct_write
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 0
 WHERE workspace_status like 'RUNNING': 0
 WHERE workspace_status like 'RUNNING%': 0
 WHERE workspace_status like '%RUNNING': 1


workspace_info,workspace_status,workspace_status_message
"List(List(false), List(false))",RUNNING,Workspace is running.


CHECKING: kuba_bootstrap_direct_write
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 1
 WHERE workspace_status like 'RUNNING': 1
 WHERE workspace_status like 'RUNNING%': 1
 WHERE workspace_status like '%RUNNING': 1


In [0]:
# We can copy the table and we get the same result
table_name_copy = f"{table_name}_copy"
spark.sql(f"DROP TABLE IF EXISTS {table_name_copy}")
spark.sql(f"CREATE TABLE {table_name_copy} AS SELECT * FROM {table_name}")
check_table(table_name_copy)
assert spark.table(table_name_copy).collect()[0].workspace_status == "RUNNING"
assert len(spark.table(table_name_copy).collect()[0].workspace_status) == len("RUNNING")
spark.table(table_name_copy).display()

CHECKING: kuba_bootstrap_copy
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 0
 WHERE workspace_status like 'RUNNING': 0
 WHERE workspace_status like 'RUNNING%': 0
 WHERE workspace_status like '%RUNNING': 1


workspace_info,workspace_status,workspace_status_message
"List(List(false), List(false))",RUNNING,Workspace is running.


In [0]:
# If we just copy the one column, it works
table_name_copy2 = f"{table_name}_copy2"
spark.sql(f"DROP TABLE IF EXISTS {table_name_copy2}")
spark.sql(f"CREATE TABLE {table_name_copy2} AS SELECT workspace_status FROM {table_name}")
check_table(table_name_copy2)
assert spark.table(table_name_copy2).collect()[0].workspace_status == "RUNNING"
assert len(spark.table(table_name_copy2).collect()[0].workspace_status) == len("RUNNING")
spark.table(table_name_copy2).display()

CHECKING: kuba_bootstrap_copy2
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 1
 WHERE workspace_status like 'RUNNING': 1
 WHERE workspace_status like 'RUNNING%': 1
 WHERE workspace_status like '%RUNNING': 1


workspace_status
RUNNING


In [0]:
# If we add a column, it still breaks
table_name_copy3 = f"{table_name}_copy3"
spark.sql(f"DROP TABLE IF EXISTS {table_name_copy3}")
spark.sql(f"CREATE TABLE {table_name_copy3} AS SELECT *, 1 as dummy FROM {table_name}")
check_table(table_name_copy3)
assert spark.table(table_name_copy3).collect()[0].workspace_status == "RUNNING"
assert len(spark.table(table_name_copy3).collect()[0].workspace_status) == len("RUNNING")
spark.table(table_name_copy3).display()

CHECKING: kuba_bootstrap_copy3
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 0
 WHERE workspace_status like 'RUNNING': 0
 WHERE workspace_status like 'RUNNING%': 0
 WHERE workspace_status like '%RUNNING': 1


workspace_info,workspace_status,workspace_status_message,dummy
"List(List(false), List(false))",RUNNING,Workspace is running.,1


In [0]:
# If we clone the table instead of copying it, it still breaks
table_name_copy4 = f"{table_name}_copy4"
spark.sql(f"DROP TABLE IF EXISTS {table_name_copy4}")
spark.sql(f"CREATE TABLE {table_name_copy4} CLONE {table_name}")
check_table(table_name_copy4)
assert spark.table(table_name_copy4).collect()[0].workspace_status == "RUNNING"
assert len(spark.table(table_name_copy4).collect()[0].workspace_status) == len("RUNNING")
spark.table(table_name_copy4).display()

CHECKING: kuba_bootstrap_copy4
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 0
 WHERE workspace_status like 'RUNNING': 0
 WHERE workspace_status like 'RUNNING%': 0
 WHERE workspace_status like '%RUNNING': 1


workspace_info,workspace_status,workspace_status_message
"List(List(false), List(false))",RUNNING,Workspace is running.


In [0]:
# If we remove the content of the other columns, it works fine
spark.sql(f"CREATE OR REPLACE TABLE {table_name_copy4} CLONE {table_name}")
spark.sql(f"UPDATE {table_name_copy4} SET workspace_info = null")
check_table(table_name_copy4)

CHECKING: kuba_bootstrap_copy4
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 1
 WHERE workspace_status like 'RUNNING': 1
 WHERE workspace_status like 'RUNNING%': 1
 WHERE workspace_status like '%RUNNING': 1


In [0]:
# If we explicitly set the workspace_status column, it works fine
spark.sql(f"CREATE OR REPLACE TABLE {table_name_copy4} CLONE {table_name}")
spark.sql(f"UPDATE {table_name_copy4} SET workspace_status = 'RUNNING'")
check_table(table_name_copy4)
spark.table(table_name_copy4).display()

CHECKING: kuba_bootstrap_copy4
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 1
 WHERE workspace_status like 'RUNNING': 1
 WHERE workspace_status like 'RUNNING%': 1
 WHERE workspace_status like '%RUNNING': 1


workspace_info,workspace_status,workspace_status_message
"List(List(false), List(false))",RUNNING,Workspace is running.


In [0]:
# If we explicitly set the workspace_status_message column, it works fine
spark.sql(f"CREATE OR REPLACE TABLE {table_name_copy4} CLONE {table_name}")
spark.sql(f"UPDATE {table_name_copy4} SET workspace_status_message = 'Workspace is running.'")
check_table(table_name_copy4)
spark.table(table_name_copy4).display()

CHECKING: kuba_bootstrap_copy4
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 1
 WHERE workspace_status like 'RUNNING': 1
 WHERE workspace_status like 'RUNNING%': 1
 WHERE workspace_status like '%RUNNING': 1


workspace_info,workspace_status,workspace_status_message
"List(List(false), List(false))",RUNNING,Workspace is running.


In [0]:
# Just redo table 4 copy again to prove it's still broken
spark.sql(f"CREATE OR REPLACE TABLE {table_name_copy4} CLONE {table_name}")
check_table(table_name_copy4)

CHECKING: kuba_bootstrap_copy4
 WHERE workspace_status : 1
 WHERE workspace_status = 'RUNNING': 0
 WHERE workspace_status like 'RUNNING': 0
 WHERE workspace_status like 'RUNNING%': 0
 WHERE workspace_status like '%RUNNING': 1
